# 📥 Ingestão de Dados IBGE – API SIDRA

## Objetivo

Extrair dados de produção agrícola municipal (**manga** e **uva**) via API SIDRA do IBGE, estruturando uma base histórica consistente para o Vale do São Francisco (Petrolina–PE e Juazeiro–BA).  

## Escopo dos Dados

- **Período:** 2013–2024  
- **Municípios:** Petrolina (PE) e Juazeiro (BA)  
- **Produtos:**  
  - Manga (código 2737)  
  - Uva (código 2748)  
- **Variáveis:**  
  - Área colhida  
  - Quantidade produzida  
  - Rendimento médio da produção  

## Fonte dos Dados

- **API:** IBGE SIDRA – Produção Agrícola Municipal (PAM, Lavouras Permanentes)  
- **Características:**  
  - Dados anuais por município e produto  
  - Acesso via requisição HTTP com parâmetros (tabela, variáveis, produtos, municípios, anos)  

## Papel deste Notebook no Projeto

Este notebook é o **primeiro estágio do pipeline**, responsável por:  
1. Conectar na API SIDRA com sessão HTTP robusta (retry e timeout).  
2. Baixar e consolidar os dados brutos em formato tabular.  
3. Salvar um CSV padronizado na pasta `data/raw` para ser usado no notebook de limpeza (`02_data_cleaning_FIXED.ipynb`).  

A preocupação aqui é **reprodutibilidade** e **confiabilidade da ingestão**, garantindo que qualquer pessoa consiga reconstruir a base bruta a partir da fonte oficial.  


In [1]:
# ============================================================================
# CÉLULA 1: IMPORTS E CONFIGURAÇÕES INICIAIS
# ============================================================================
# Organização:
# - Standard library
# - Third-party
# - Colab / utilitários
# ============================================================================

# --- Standard library ---
import os
import logging
from typing import Dict, Any
from urllib3.util.retry import Retry

# --- Third-party ---
import pandas as pd
import requests
from requests.adapters import HTTPAdapter

# --- Google Colab / utilitários ---
from google.colab import drive
from IPython.display import display

# --- Configuração de logging ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

logger.info("Inicializando notebook 01_data_ingestion (IBGE SIDRA).")

# --- Montagem do Google Drive ---
drive.mount("/content/drive")

# Diretórios base do projeto
BASE_DIR = "/content/drive/MyDrive/projeto_produtividade"
RAW_DATA_DIR = os.path.join(BASE_DIR, "data", "raw")

os.makedirs(RAW_DATA_DIR, exist_ok=True)
logger.info(f"Diretório de dados RAW garantido em: {RAW_DATA_DIR}")

Mounted at /content/drive


## ⚙️ CÉLULA 2 – Parâmetros finais da consulta SIDRA

Nesta célula definimos, de forma explícita e documentada, **todos os parâmetros** usados na chamada à API SIDRA do IBGE:  

- **Tabela:** 1613 – Produção Agrícola Municipal (Lavouras Permanentes).  
- **Nível geográfico:** Municípios (`n6`).  
- **Municípios-alvo:**  
  - 2611101 – Petrolina (PE)  
  - 2918407 – Juazeiro (BA)  
- **Período:** 2013–2024 (série histórica usada no MVP).  
- **Variáveis:**  
  - 216 – Área colhida  
  - 214 – Quantidade produzida  
  - 112 – Rendimento médio da produção  
- **Classificação (c82 – Lavouras permanentes):**  
  - 2748 – Uva  
  - 2737 – Manga  

Também definimos o caminho de saída para o arquivo bruto `pam_permanente_vsf_raw.csv` na pasta `data/raw`, que alimenta o notebook de limpeza (`02_data_cleaning_FIXED.ipynb`).  


In [2]:
# ============================================================================
# CÉLULA 2: PARÂMETROS FINAIS DA CONSULTA À API SIDRA
# ============================================================================

# --- PARÂMETROS DA TABELA E GEOGRAFIA ---

# Tabela 1613: Produção Agrícola Municipal - Lavouras PERMANENTES
TABLE_CODE = "1613"

# Nível geográfico 6: Municípios
GEOGRAPHIC_LEVEL = "n6"

# Códigos IBGE dos municípios do Vale do São Francisco (PE e BA)
MUNICIPIOS_VSF = [
    "2611101",  # Petrolina (PE)
    "2918407",  # Juazeiro (BA)
]
GEO_CODES = ",".join(MUNICIPIOS_VSF)

# --- PERÍODO E VARIÁVEIS ---

# Período de análise (série histórica 2013–2024)
PERIOD = "2013-2024"

# Variáveis de interesse:
# 216: Área colhida
# 214: Quantidade produzida
# 112: Rendimento médio da produção
VARIABLES = "216,214,112"

# --- CLASSIFICAÇÃO DE PRODUTOS (LAVOURAS PERMANENTES) ---

# Classificação c82 (Lavouras Permanentes) e códigos:
# 2748: Uva
# 2737: Manga
CLASSIFICATION_PATH = "c82/2748,2737"

# --- CAMINHOS DE ARQUIVOS ---

BASE_DIR = "/content/drive/MyDrive/projeto_produtividade"
OUTPUT_DIR = os.path.join(BASE_DIR, "data", "raw")
OUTPUT_FILE_PATH = os.path.join(OUTPUT_DIR, "pam_permanente_vsf_raw.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

logger.info("Parâmetros FINAIS configurados:")
logger.info(f"Tabela-alvo (TABLE_CODE): {TABLE_CODE}")
logger.info(f"Nível geográfico (GEOGRAPHIC_LEVEL): {GEOGRAPHIC_LEVEL}")
logger.info(f"Municípios (GEO_CODES): {GEO_CODES}")
logger.info(f"Período (PERIOD): {PERIOD}")
logger.info(f"Variáveis (VARIABLES): {VARIABLES}")
logger.info(f"Classificação (CLASSIFICATION_PATH): {CLASSIFICATION_PATH}")
logger.info(f"Arquivo de saída (OUTPUT_FILE_PATH): {OUTPUT_FILE_PATH}")

## 🌐 CÉLULA 3 – Workflow completo de ingestão via API SIDRA

Aqui implementamos o fluxo completo de ingestão:

1. Montar a URL da API SIDRA com base nos parâmetros da CÉLULA 2.  
2. Criar uma sessão HTTP com **retry** e **timeout** configurados.  
3. Executar a requisição, tratar erros e converter a resposta JSON em `DataFrame`.  
4. Salvar os dados brutos em `pam_permanente_vsf_raw.csv` na pasta `data/raw`.  
5. Exibir uma amostra dos dados para validação rápida.  

Essa célula é o “botão de executar” da ingestão: ao rodá-la, toda a coleta de dados IBGE é refeita de ponta a ponta.  

In [3]:
# ============================================================================
# CÉLULA 3: WORKFLOW COMPLETO DE INGESTÃO (FINAL REFACTORED)
# ============================================================================

def create_sidra_session() -> requests.Session:
    """
    Cria uma sessão HTTP com política de retry para chamadas à API SIDRA.
    """
    session = requests.Session()
    retries = Retry(
        total=5,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session


def build_sidra_url() -> str:
    """
    Monta a URL completa da API SIDRA com base nos parâmetros globais.
    """
    base_url = "https://apisidra.ibge.gov.br/values"
    url = (
        f"{base_url}/t/{TABLE_CODE}/{GEOGRAPHIC_LEVEL}/{GEO_CODES}/"
        f"p/{PERIOD}/v/{VARIABLES}/{CLASSIFICATION_PATH}"
    )
    return url


def run_ingestion() -> pd.DataFrame:
    """
    Executa o workflow de ingestão:
    - Monta URL
    - Faz requisição com retry
    - Converte JSON em DataFrame
    - Salva CSV em OUTPUT_FILE_PATH
    """
    url = build_sidra_url()
    logger.info("Executando requisição para a API do IBGE SIDRA...")
    logger.info(f"URL: {url}")

    session = create_sidra_session()

    try:
        resp = session.get(url, timeout=90)
        resp.raise_for_status()
        data = resp.json()
        logger.info("Dados recebidos com sucesso da API SIDRA.")

        if not data or len(data) <= 1:
            logger.warning(
                "A API retornou resposta vazia ou apenas cabeçalho. "
                "Verifique parâmetros de tabela, período ou classificação."
            )
            return pd.DataFrame()

        # Primeira linha do JSON é o cabeçalho
        header = data[0]
        df_raw = pd.DataFrame(data[1:])
        df_raw.columns = header.values()
        logger.info("DataFrame criado com sucesso a partir da resposta da API.")

        # Salvar CSV bruto
        df_raw.to_csv(OUTPUT_FILE_PATH, index=False, encoding="utf-8-sig")
        logger.info(f"SUCESSO: Dados brutos salvos em: {OUTPUT_FILE_PATH}")

        return df_raw

    except requests.exceptions.HTTPError as err:
        msg = f"Erro HTTP na requisição: {err.response.status_code}\n{err.response.text[:500]}"
        logger.error(msg)
        raise RuntimeError(msg) from err
    except Exception as e:
        msg = f"Ocorreu um erro inesperado durante o processo de ingestão: {e}"
        logger.error(msg)
        raise RuntimeError(msg) from e


# --- EXECUÇÃO DA INGESTÃO ---

df_ibge_raw = run_ingestion()

if not df_ibge_raw.empty:
    print("\nAmostra dos dados brutos retornados pela API:")
    display(df_ibge_raw.head())
else:
    print("\n⚠️ Nenhum dado foi retornado. Verifique os logs acima.")


Amostra dos dados brutos retornados pela API:


,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável,Produto das lavouras permanentes (Código),Produto das lavouras permanentes
0,6,Município,1006,Hectares,4642,2611101,Petrolina (PE),2013,2013,216,Área colhida,2748,Uva
1,6,Município,1006,Hectares,7880,2611101,Petrolina (PE),2013,2013,216,Área colhida,2737,Manga
2,6,Município,1017,Toneladas,162448,2611101,Petrolina (PE),2013,2013,214,Quantidade produzida,2748,Uva
3,6,Município,1017,Toneladas,173360,2611101,Petrolina (PE),2013,2013,214,Quantidade produzida,2737,Manga
4,6,Município,33,Quilogramas por Hectare,34995,2611101,Petrolina (PE),2013,2013,112,Rendimento médio da produção,2748,Uva
